# Example 3: Load Concepts and Demonstrate Neuron Manipulation

This notebook demonstrates how to:
1. Load the language model and trained SAE from previous examples
2. Load curated concepts from the manual curation process
3. Attach the concept dictionary to the SAE
4. Demonstrate inference with manipulated neurons using the concepts

This example shows how to use the curated concepts to understand and manipulate what each neuron represents in the model.


In [ ]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
import json
import csv
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter

from amber.store import LocalStore
from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from amber.mechanistic.autoencoder.autoencoder import Autoencoder
from amber.mechanistic.autoencoder.concepts.concept_dictionary import ConceptDictionary, Concept

print("✅ Imports completed")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Imports completed


In [65]:
# Configuration
print("🚀 Starting Concept Loading and Neuron Manipulation Example")

# Load metadata from previous examples
training_metadata_path = Path("outputs/training_metadata.json")
attachment_metadata_path = Path("outputs/attachment_metadata.json")

if not training_metadata_path.exists():
    print("❌ Error: training_metadata.json not found!")
    print("   Please run 01_train_sae_model.ipynb first")
    raise FileNotFoundError("training_metadata.json not found")

if not attachment_metadata_path.exists():
    print("❌ Error: attachment_metadata.json not found!")
    print("   Please run 02_attach_sae_and_save_texts.ipynb first")
    raise FileNotFoundError("attachment_metadata.json not found")

# Load metadata
with open(training_metadata_path, "r") as f:
    training_metadata = json.load(f)

with open(attachment_metadata_path, "r") as f:
    attachment_metadata = json.load(f)

# Configuration from metadata
MODEL_ID = training_metadata["model_id"]
LAYER_SIGNATURE = training_metadata["layer_signature"]
SAE_MODEL_PATH = Path(training_metadata["sae_model_path"])
CACHE_DIR = Path(training_metadata["cache_dir"])
STORE_DIR = Path(training_metadata["store_dir"])
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Check for curated concepts
CURATED_CONCEPTS_CSV = Path("outputs/curated_concepts.csv")
CURATED_CONCEPTS_JSON = Path("outputs/curated_concepts.json")

if not CURATED_CONCEPTS_CSV.exists() and not CURATED_CONCEPTS_JSON.exists():
    print("⚠️ Warning: No curated concepts found!")
    print("   Please run the manual curation process first")
    print("   You can create a simple CSV with format: neuron_idx,concept_name,score")

print(f"🔧 Model: {MODEL_ID}")
print(f"🎯 Target layer: {LAYER_SIGNATURE}")
print(f"🧠 SAE model: {SAE_MODEL_PATH}")
print(f"📊 Curated concepts: {CURATED_CONCEPTS_CSV if CURATED_CONCEPTS_CSV.exists() else CURATED_CONCEPTS_JSON}")
print()


🚀 Starting Concept Loading and Neuron Manipulation Example
🔧 Model: sshleifer/tiny-gpt2
🎯 Target layer: gpt2lmheadmodel_transformer_h_0_attn_c_attn
🧠 SAE model: outputs/sae_model.pt
📊 Curated concepts: outputs/curated_concepts.csv



In [ ]:
# Step 1: Load language model
print("📥 Loading language model...")

# Load model and move to device
model = LanguageModel.from_huggingface(MODEL_ID)
model.model.to(DEVICE)

# Optional: set experiment metadata
model.context.experiment_name = "sae_attachment"
model.context.run_id = f"attachment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
model.context.max_length = 64

print(f"✅ Model loaded: {model.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"🔧 Context: {model.context.experiment_name}/{model.context.run_id}")


📥 Loading language model...
✅ Model loaded: sshleifer/tiny-gpt2
📱 Device: cpu
🔧 Context: sae_attachment/attachment_20251026_222155


In [ ]:
# Step 2: Load trained SAE
print("📥 Loading trained SAE...")
if not SAE_MODEL_PATH.exists():
    print(f"❌ Error: SAE model not found at {SAE_MODEL_PATH}")
    print("   Please run 01_train_sae_model.ipynb first")
    raise FileNotFoundError(f"SAE model not found at {SAE_MODEL_PATH}")

sae, dataset_normalize, dataset_target_norm, dataset_mean = Autoencoder.load_model(SAE_MODEL_PATH)
sae.to(DEVICE)

# Update SAE context with current experiment info
sae.context.experiment_name = "concept_manipulation"
sae.context.run_id = f"manipulation_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

print(
    f"✅ SAE loaded: {training_metadata['hidden_dim']} → {training_metadata['n_latents']} → {training_metadata['hidden_dim']}")
print(f"🔧 Context: {sae.context.experiment_name}/{sae.context.run_id}")
print(f"📊 Dataset normalization: {dataset_normalize}")
print("✅ Trained SAE loaded")


2025-10-26 22:21:56,785 [INFO] amber.mechanistic.autoencoder.autoencoder: 
Loaded model from outputs/sae_model.pt
n_latents=24, n_inputs=6, activation=TopK_8, tied=False


📥 Loading trained SAE...
✅ SAE loaded: 6 → 24 → 6
🔧 Context: concept_manipulation/manipulation_20251026_222156
📊 Dataset normalization: False
✅ Trained SAE loaded


In [68]:
# Step 3: Load curated concepts
print("📥 Loading curated concepts...")

# Try to load from CSV first, then JSON
if CURATED_CONCEPTS_CSV.exists():
    print(f"📄 Loading from CSV: {CURATED_CONCEPTS_CSV}")
    concept_dict = ConceptDictionary.from_csv(CURATED_CONCEPTS_CSV, n_size=training_metadata["n_latents"])
elif CURATED_CONCEPTS_JSON.exists():
    print(f"📄 Loading from JSON: {CURATED_CONCEPTS_JSON}")
    concept_dict = ConceptDictionary.from_directory(CURATED_CONCEPTS_JSON.parent)
else:
    print("⚠️ No curated concepts found, creating example concepts...")
    # Create some example concepts for demonstration
    concept_dict = ConceptDictionary(n_size=training_metadata["n_latents"], max_concepts=5)

    # Add some example concepts
    concept_dict.add(0, "family relationships", 0.9)
    concept_dict.add(0, "parent-child interactions", 0.8)
    concept_dict.add(1, "nature and outdoors", 0.9)
    concept_dict.add(1, "animals and wildlife", 0.8)
    concept_dict.add(2, "problem solving", 0.9)
    concept_dict.add(2, "logical thinking", 0.8)
    concept_dict.add(3, "emotional expressions", 0.9)
    concept_dict.add(3, "feelings and moods", 0.8)

print(f"✅ Loaded concept dictionary with {concept_dict.n_size} neurons")
print(f"📊 Total concepts: {sum(len(concept_dict.get(i)) for i in range(concept_dict.n_size))}")

# Show some concepts
print("\n🔍 Sample concepts:")
for neuron_idx in range(min(5, concept_dict.n_size)):
    concepts = concept_dict.get(neuron_idx)
    if concepts:
        print(f"   Neuron {neuron_idx}: {len(concepts)} concepts")
        for i, concept in enumerate(concepts[:2]):
            print(f"     {i + 1}. '{concept.name}' (score: {concept.score:.3f})")
    else:
        print(f"   Neuron {neuron_idx}: no concepts")
print()


📥 Loading curated concepts...
📄 Loading from CSV: outputs/curated_concepts.csv
✅ Loaded concept dictionary with 24 neurons
📊 Total concepts: 60

🔍 Sample concepts:
   Neuron 0: 10 concepts
     1. 'family relationships' (score: 0.900)
     2. 'parent-child interactions' (score: 0.800)
   Neuron 1: 10 concepts
     1. 'nature and outdoors' (score: 0.900)
     2. 'animals and wildlife' (score: 0.800)
   Neuron 2: 10 concepts
     1. 'problem solving' (score: 0.900)
     2. 'logical thinking' (score: 0.800)
   Neuron 3: 10 concepts
     1. 'Mummy and Daddy were picking flowers in the garden. Mummy picked a red daisy, Daddy picked a purple ' (score: 0.006)
     2. 'Once there was a generous bear. He liked to help others and was always very kind. But he had one hab' (score: 0.006)
   Neuron 4: 10 concepts
     1. 'Mummy and Daddy were picking flowers in the garden. Mummy picked a red daisy, Daddy picked a purple ' (score: 0.062)
     2. 'Once there was a generous bear. He liked to help othe

In [ ]:
# Step 4: Attach concept dictionary to SAE
print("🔗 Attaching concept dictionary to SAE...")

# Set the language model and layer signature on the SAE's context
sae.context.lm = model
sae.context.lm_layer_signature = LAYER_SIGNATURE

# Attach the concept dictionary to the SAE
sae.concepts.dictionary = concept_dict

print("✅ Concept dictionary attached to SAE")
print(f"🔧 SAE now has access to {sum(len(concept_dict.get(i)) for i in range(concept_dict.n_size))} concepts")
print(f"📊 Concepts available for {sum(1 for i in range(concept_dict.n_size) if concept_dict.get(i))} neurons")

🔗 Attaching concept dictionary to SAE...
✅ Concept dictionary attached to SAE
🔧 SAE now has access to 60 concepts
📊 Concepts available for 6 neurons


In [72]:
test_texts = [
    "The family went to the park together.",
    "The cat sat on the tree branch.",
    "She solved the math problem quickly.",
    "The child felt happy and excited."
]

In [70]:
# Step 5: Demonstrate neuron manipulation using sae.concepts methods
print("🎯 Demonstrating neuron manipulation using sae.concepts methods...")

# Test texts for inference


print(f"📝 Test texts:")
for i, text in enumerate(test_texts):
    print(f"   {i+1}. \"{text}\"")
print()

# Run inference to get baseline activations
print("🔍 Running baseline inference...")
with torch.no_grad():
    baseline_output, baseline_activations = model.forwards(test_texts)

    # Get SAE activations for the target layer
    sae_input = baseline_activations[LAYER_SIGNATURE]
    sae_output = sae(sae_input)
    baseline_sae_activations = sae_output[1]  # Get latent activations

print(f"✅ Baseline inference completed")
print(f"📊 SAE activations shape: {baseline_sae_activations.shape}")
print()

# Show which neurons are most active for each text
print("🧠 Most active neurons for each text:")
for i, text in enumerate(test_texts):
    activations = baseline_sae_activations[i]
    top_neurons = torch.topk(activations, k=3)

    print(f"   Text {i+1}: \"{text}\"")
    for j, (neuron_idx, activation) in enumerate(zip(top_neurons.indices, top_neurons.values)):
        concepts = concept_dict.get(neuron_idx.item())
        concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
        print(f"     {j+1}. Neuron {neuron_idx.item()}: {activation:.4f} - {concept_names}")
    print()


🎯 Demonstrating neuron manipulation using sae.concepts methods...
📝 Test texts:
   1. "The family went to the park together."
   2. "The cat sat on the tree branch."
   3. "She solved the math problem quickly."
   4. "The child felt happy and excited."

🔍 Running baseline inference...


KeyError: 'gpt2lmheadmodel_transformer_h_0_attn_c_attn'

In [ ]:
# Step 6: Concept manipulation using manipulate_concept method
print("🔧 Demonstrating concept manipulation using manipulate_concept method...")

# Method: Using manipulate_concept to boost/suppress specific concepts
print("\n📈 Using manipulate_concept to manipulate concepts")

# Reset multiplication to 1.0 for all neurons
sae.concepts.multiplication.data.fill_(1.0)

# Boost family-related concepts (neuron 0)
family_boost = 2.0
sae.concepts.manipulate_concept(0, multiplier=family_boost)
print(f"   🎯 Boosting neuron 0 (family concepts) by {family_boost}x using manipulate_concept")

# Boost nature-related concepts (neuron 1) 
nature_boost = 3.0
sae.concepts.manipulate_concept(1, multiplier=nature_boost)
print(f"   🎯 Boosting neuron 1 (nature concepts) by {nature_boost}x using manipulate_concept")

# Suppress problem-solving concepts (neuron 2)
problem_suppress = 0.5
sae.concepts.manipulate_concept(2, multiplier=problem_suppress)
print(f"   🎯 Suppressing neuron 2 (problem concepts) by {problem_suppress}x using manipulate_concept")

# Boost emotional concepts (neuron 3)
emotional_boost = 1.5
sae.concepts.manipulate_concept(3, multiplier=emotional_boost)
print(f"   🎯 Boosting neuron 3 (emotional concepts) by {emotional_boost}x using manipulate_concept")

# Run inference with manipulated concepts
print("\n🔍 Running inference with manipulate_concept manipulation...")
with torch.no_grad():
    manipulated_output, manipulated_activations = model.forwards(test_texts)
    
    # Get SAE activations for the target layer
    sae_input = manipulated_activations[LAYER_SIGNATURE]
    sae_output = sae(sae_input)
    manipulated_sae_activations = sae_output[1]  # Get latent activations

print(f"✅ manipulate_concept manipulation inference completed")

# Compare original vs manipulated activations
print("\n📊 Comparison: Original vs manipulate_concept Manipulated")
for i, text in enumerate(test_texts):
    print(f"\n   Text {i+1}: \"{text}\"")
    
    # Original activations
    original_activations = baseline_sae_activations[i]
    top_original = torch.topk(original_activations, k=3)
    
    print(f"   📈 Original top neurons:")
    for j, (neuron_idx, activation) in enumerate(zip(top_original.indices, top_original.values)):
        concepts = concept_dict.get(neuron_idx.item())
        concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
        print(f"     {j+1}. Neuron {neuron_idx.item()}: {activation:.4f} - {concept_names}")
    
    # Manipulated activations
    manipulated_activations_text = manipulated_sae_activations[i]
    top_manipulated = torch.topk(manipulated_activations_text, k=3)
    
    print(f"   🚀 manipulate_concept manipulated top neurons:")
    for j, (neuron_idx, activation) in enumerate(zip(top_manipulated.indices, top_manipulated.values)):
        concepts = concept_dict.get(neuron_idx.item())
        concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
        print(f"     {j+1}. Neuron {neuron_idx.item()}: {activation:.4f} - {concept_names}")
    
    # Show changes
    activation_changes = (manipulated_activations_text - original_activations).abs()
    top_changes = torch.topk(activation_changes, k=3)
    print(f"   🔄 Biggest changes:")
    for j, (neuron_idx, change) in enumerate(zip(top_changes.indices, top_changes.values)):
        concepts = concept_dict.get(neuron_idx.item())
        concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
        print(f"     {j+1}. Neuron {neuron_idx.item()}: +{change:.4f} - {concept_names}")


🔧 Demonstrating concept manipulation using multiply_concept method...

📈 Using multiply_concept to manipulate concepts
   🎯 Boosting neuron 0 (family concepts) by 2.0x using multiply_concept
   🎯 Boosting neuron 1 (nature concepts) by 3.0x using multiply_concept
   🎯 Suppressing neuron 2 (problem concepts) by 0.5x using multiply_concept
   🎯 Boosting neuron 3 (emotional concepts) by 1.5x using multiply_concept

🔍 Running inference with multiply_concept manipulation...


KeyError: 'gpt2lmheadmodel_transformer_h_0_attn_c_attn'

In [ ]:
# Step 7: Advanced manipulate_concept scenarios
print("\n🔧 Advanced manipulate_concept scenarios")

# Reset multiplication to 1.0 for all neurons
sae.concepts.multiplication.data.fill_(1.0)

# Scenario 1: Boost multiple related concepts
print("\n📈 Scenario 1: Boost multiple related concepts")
sae.concepts.manipulate_concept(0, multiplier=2.5)  # Family concepts
sae.concepts.manipulate_concept(1, multiplier=2.0)  # Nature concepts
print("   🎯 Boosted family and nature concepts together")

# Run inference for scenario 1
print("\n🔍 Running inference for scenario 1...")
with torch.no_grad():
    scenario1_output, scenario1_activations = model.forwards(test_texts)
    sae_input = scenario1_activations[LAYER_SIGNATURE]
    sae_output = sae(sae_input)
    scenario1_sae_activations = sae_output[1]

# Scenario 2: Suppress specific concepts
print("\n📉 Scenario 2: Suppress specific concepts")
sae.concepts.multiplication.data.fill_(1.0)  # Reset
sae.concepts.manipulate_concept(2, multiplier=0.3)  # Suppress problem-solving
sae.concepts.manipulate_concept(3, multiplier=0.4)  # Suppress emotional
print("   🎯 Suppressed problem-solving and emotional concepts")

# Run inference for scenario 2
print("\n🔍 Running inference for scenario 2...")
with torch.no_grad():
    scenario2_output, scenario2_activations = model.forwards(test_texts)
    sae_input = scenario2_activations[LAYER_SIGNATURE]
    sae_output = sae(sae_input)
    scenario2_sae_activations = sae_output[1]

# Scenario 3: Mixed boost/suppress
print("\n🎛️ Scenario 3: Mixed boost/suppress")
sae.concepts.multiplication.data.fill_(1.0)  # Reset
sae.concepts.manipulate_concept(0, multiplier=3.0)  # Strong boost family
sae.concepts.manipulate_concept(1, multiplier=0.2)  # Strong suppress nature
sae.concepts.manipulate_concept(2, multiplier=1.5)  # Moderate boost problem-solving
sae.concepts.manipulate_concept(3, multiplier=0.8)  # Slight suppress emotional
print("   🎯 Mixed manipulation: boost family/problem, suppress nature/emotional")

# Run inference for scenario 3
print("\n🔍 Running inference for scenario 3...")
with torch.no_grad():
    scenario3_output, scenario3_activations = model.forwards(test_texts)
    sae_input = scenario3_activations[LAYER_SIGNATURE]
    sae_output = sae(sae_input)
    scenario3_sae_activations = sae_output[1]

print(f"✅ All manipulate_concept scenarios completed")

# Show results for each scenario
print("\n📊 Results Summary:")
scenarios = [
    ("Baseline", baseline_sae_activations),
    ("Scenario 1: Boost family+nature", scenario1_sae_activations),
    ("Scenario 2: Suppress problem+emotional", scenario2_sae_activations),
    ("Scenario 3: Mixed manipulation", scenario3_sae_activations)
]

for scenario_name, activations in scenarios:
    print(f"\n   {scenario_name}:")
    for i, text in enumerate(test_texts):
        text_activations = activations[i]
        top_neurons = torch.topk(text_activations, k=2)
        
        print(f"     Text {i+1}: \"{text[:30]}...\"")
        for j, (neuron_idx, activation) in enumerate(zip(top_neurons.indices, top_neurons.values)):
            concepts = concept_dict.get(neuron_idx.item())
            concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
            print(f"       {j+1}. Neuron {neuron_idx.item()}: {activation:.4f} - {concept_names}")


In [ ]:
# Step 8: multiply_concept parameter inspection and summary
print("\n🔧 multiply_concept parameter inspection and summary")

# Show current concept manipulation parameters
print("\n📊 Current multiply_concept parameters:")
print("   🎛️ Multiplication parameters:")
for i in range(min(8, sae.concepts.multiplication.shape[0])):
    concepts = concept_dict.get(i)
    concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
    mult_val = sae.concepts.multiplication.data[i].item()
    print(f"     Neuron {i}: {mult_val:.2f}x - {concept_names}")

# Demonstrate resetting parameters
print("\n🔄 Resetting all multiply_concept parameters to 1.0...")
sae.concepts.multiplication.data.fill_(1.0)

print("   ✅ All parameters reset to 1.0 (no manipulation)")

# Show reset parameters
print("\n📊 Reset multiply_concept parameters:")
for i in range(min(8, sae.concepts.multiplication.shape[0])):
    concepts = concept_dict.get(i)
    concept_names = [c.name for c in concepts] if concepts else ["no concepts"]
    mult_val = sae.concepts.multiplication.data[i].item()
    print(f"     Neuron {i}: {mult_val:.2f}x - {concept_names}")

print("\n✅ multiply_concept demonstration completed!")
print("💡 This shows how you can use multiply_concept to:")
print("   1. Boost specific concepts by multiplying activations (multiplier > 1.0)")
print("   2. Suppress specific concepts by reducing activations (multiplier < 1.0)")
print("   3. Apply different multipliers to different neurons simultaneously")
print("   4. Reset all parameters to 1.0 to return to baseline behavior")
print("   5. Use this for fine-grained control of model behavior based on concepts")
print("   6. Manipulate model outputs for interpretability and controlled generation")
